## 4.1 LLM架构

In [ ]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 1024, # Context length
    "emb_dim": 768,         # Embedding dimension
    "n_heads": 12,          # Number of attention heads
    "n_layers": 12,         # Number of layers
    "drop_rate": 0.1,       # Dropout rate
    "qkv_bias": False       # Query-Key-Value bias
}
#初始化定义需要的各种超参数

In [ ]:
import torch
import torch.nn as nn

class DummyGPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        # 词嵌入层，将输入索引转换为词向量，词表大小由字典大小和特征维度决定。
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        # 位置信息嵌入层，基于文本长度和特征维度生成位置信息。
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        # Dropout 层，用于随机丢弃一部分嵌入信息以减少过拟合。
        self.drop_emb = nn.Dropout(cfg["drop_rate"])

        # 使用多个 Transformer 块（占位符）
        # Transformer 模块的堆叠，模型核心部分。
        self.trf_blocks = nn.Sequential(
            *[DummyTransformerBlock(cfg) for _ in range(cfg["n_layers"])]
        )

        # 使用归一化层（占位符）
        # 最终归一化层，用于调整特征分布。
        self.final_norm = DummyLayerNorm(cfg["emb_dim"])

        # 输出层，将特征映射到词表分布，最终预测输出单词。
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )

    def forward(self, in_idx):
        # 获取批次大小和序列长度。
        batch_size, seq_len = in_idx.shape

        # 根据输入索引生成词嵌入。
        tok_embeds = self.tok_emb(in_idx)
        # 生成对应的位置信息嵌入。
        # arange 生成一个从0到seq_len-1的序列，device=in_idx.device 指定设备。
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))

        # 将词嵌入和位置信息嵌入相加。
        x = tok_embeds + pos_embeds
        # 应用 Dropout 随机丢弃部分信息。
        x = self.drop_emb(x)
        # 通过多个 Transformer 块处理特征。
        x = self.trf_blocks(x)
        # 应用最终的归一化层。
        x = self.final_norm(x)
        # 将隐藏状态映射到词表分布，生成预测结果。
        logits = self.out_head(x)

        return logits
    
class DummyTransformerBlock(nn.Module):
    # Transformer 块的占位类。
    def __init__(self, cfg):
        super().__init__()
        # 占位，实际模型应实现注意力机制和前馈网络。
    
    def forward(self, x):
        # 此块不执行任何操作，仅返回输入。
        return x

class DummyLayerNorm(nn.Module):
    # 归一化层的占位类。
    def __init__(self, normalized_shape, eps=1e-5):
        super().__init__()
        # 参数用于模拟 LayerNorm 的接口。
    
    def forward(self, x):
        # 此层不执行任何操作，仅返回输入。
        return x

In [ ]:
import tiktoken

# 使用gpt2的分词器
tokenizer = tiktoken.get_encoding("gpt2")

batch = []

txt1 = "Every effort moves you"
txt2 = "Every day holds a"

# tokenize
batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))

# 构成batch
batch = torch.stack(batch, dim=0)

print(batch)


In [ ]:
# 试运行一下模型
torch.manual_seed(123)
model = DummyGPTModel(GPT_CONFIG_124M)

logits = model(batch)
print("Output shape:", logits.shape)
print(logits)